# Spylon Kernel Test with Spark 3.4.0

This has been updated from Spark 2.4. I use a local SBT installation via /misc/build/0/classes. This is similar to the PySpark spark0 notebook.

This must use the same Scala version as Spark - which is 2.13 (it was 2.11).

I haven't recompiled the Scala source code in src - the artikus.spark classes.

Once a Spark context is instantiated, it should be accessible from http://j1:4040 if the host of this notebook is j1. This hostname is spark.driver.host

In [ ]:
%%python
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Of no use for a Spylon notebook

## Configuration and Initialization of Spark

Note that we can set things like driver memory etc.

If `launcher._spark_home` is not set it will default to looking at the `SPARK_HOME` environment variable.

I run on a cluster owned by the hadoop user who is a member of my group devel.

I build new features for Scala and access them via /misc/build/0/classes. I have to restart the kernel to access any new classes. And must relaunch Spark to access changes.

I can't change the spark.sql.warehouse.dir

In [1]:
%%init_spark
launcher.num_executors = 4
launcher.executor_cores = 4
launcher.driver_memory = '4g'
launcher.conf.set("spark.driver.cores", 4);
launcher.conf.set("spark.executor.cores", 4);
launcher.conf.set("spark.executor.memory", "4g");
launcher.conf.set("spark.executor.instances", 4);
launcher.conf.set("spark.sql.warehouse.dir", "file:///home/hadoop/data/hive");
launcher.conf.set("spark.sql.catalogImplementation", "hive");
launcher.conf.set("spark.hadoop.fs.permissions.umask-mode", "002");
launcher.conf.set("spark.driver.extraClassPath", ":/misc/build/0/classes/:/usr/share/java/postgresql.jar");

## Spark Configuration

Some basic operations.


In [2]:
spark // spark is the SQL session

Intitializing Scala interpreter ...

Spark Web UI available at http://j1.host0:4040
SparkContext available as 'sc' (version = 3.4.0, master = local[*], app id = local-1684141886448)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@2fe24383


In [3]:
spark.version

res1: String = 3.4.0


In [4]:
spark.conf.getAll foreach (x => println(x._1 + " --> " + x._2))

spark.sql.warehouse.dir --> file:/home/hadoop/data/hive
spark.hadoop.fs.permissions.umask-mode --> 002
spark.executor.extraJavaOptions --> -Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false
spark.driver.host --> j1.host0
spark.serializer.objectSt

In [5]:
var dbs1 = spark.catalog.listDatabases()
dbs1.show
spark.catalog.listCatalogs().show()

+-------+-------------+--------------------+--------------------+
|   name|      catalog|         description|         locationUri|
+-------+-------------+--------------------+--------------------+
|default|spark_catalog|Default Hive data...|file:/misc/build/...|
+-------+-------------+--------------------+--------------------+

+-------------+-----------+
|         name|description|
+-------------+-----------+
|spark_catalog|       null|
+-------------+-----------+



dbs1: org.apache.spark.sql.Dataset[org.apache.spark.sql.catalog.Database] = [name: string, catalog: string ... 2 more fields]


In [6]:
val d0 = spark.catalog.listDatabases().take(1)
d0(0).locationUri

d0: Array[org.apache.spark.sql.catalog.Database] = Array(Database[name='default', catalog='spark_catalog', description='Default Hive database', path='file:/misc/build/0/spark-eg0/spark-warehouse'])
res4: String = file:/misc/build/0/spark-eg0/spark-warehouse


In [20]:
var df0 = spark.sql("show databases")
df0.show()
df0 = spark.sql("show tables")
df0.show()
df0 = spark.sql("select count(*) from finalTable")

+---------+
|namespace|
+---------+
|  default|
+---------+

+---------+----------+-----------+
|namespace| tableName|isTemporary|
+---------+----------+-----------+
|  default|finaltable|      false|
+---------+----------+-----------+



df0: org.apache.spark.sql.DataFrame = [count(1): bigint]
df0: org.apache.spark.sql.DataFrame = [count(1): bigint]
df0: org.apache.spark.sql.DataFrame = [count(1): bigint]


## Using local Scala Builds

In [2]:
import artikus.spark.U

Intitializing Scala interpreter ...

Spark Web UI available at http://j1.host0:4040
SparkContext available as 'sc' (version = 3.4.0, master = local[*], app id = local-1684091628830)
SparkSession available as 'spark'


import artikus.spark.U


In [3]:
val cl = spark.getClass().getClassLoader()
cl.asInstanceOf[java.net.URLClassLoader].getURLs.map(x => x.toString())

java.lang.ClassCastException:  class jdk.internal.loader.ClassLoaders$AppClassLoader cannot be cast to class java.net.URLClassLoader (jdk.internal.loader.ClassLoaders$AppClassLoader and java.net.URLClassLoader are in module java.base of loader 'bootstrap')

In [4]:
// These are from the /misc/build/0/classes
U.identity
U.printClass(spark)
U.alert("hello")

class org.apache.spark.sql.SparkSession
hello


In [ ]:
U.classes(spark)

In [ ]:
U.flist(".")

## SparkSession operations

Basic operations
https://jaceklaskowski.gitbooks.io/mastering-spark-sql/spark-sql-SparkSession.html#createDataset

In [ ]:
val strings = spark.emptyDataset[String]
strings.printSchema

In [ ]:
val one = spark.createDataset(Seq(1))
one.show
one.printSchema

In [ ]:
// Use an implicit requires a "spark" in the namespace.
import spark.implicits._

val one = Seq(1).toDS
one.show
one.printSchema

In [ ]:
// Using spark.range()
val range0 = spark.range(start = 0, end = 4, step = 2, numPartitions = 5)
range0.show

In [ ]:
// More packing

In [ ]:
val sc = spark.sparkContext

In [ ]:
val data = Seq("a", "b", "c", "d") zip (0 to 4)

U.printClass(data)

In [ ]:
val data = Seq("foo", "bar", "baz") zip 1 :: 2 :: 3 :: Nil
val data1 = Seq("foo", "bar", "bar") zip 4 :: 5 :: 6 :: Nil

In [ ]:
val ds = spark.createDataset(data)

val ds1 = sc.parallelize(data)

U.printClass(ds)
U.printClass(ds1)

val ds2 = sc.parallelize(data1)

ds1.join(ds2).take(5)

In [ ]:
// Local file URI
// non-existent file loads
// /misc/build/0/prog-scala-2nd-ed-code-examples
val local2 = U.local1(".")

In [ ]:
val f1 = "rev-users.csv"
val file = sc.textFile(local2(f1).toString())
U.printClass(file)

In [ ]:
// This file has a header row
// Take the first row, index into it, split and return a sequence
val h2 = file.take(1)(0).split(",").toSeq

// Get the remainder by using subtract
// convert the header row back to an RDD using parallelize
val r1 = file.subtract(sc.parallelize(file.take(1)))

In [ ]:
// Look at the underlying row
r1.take(1)

In [ ]:
// Now map over the quantities
// The transformations are only applied when we take(), use the column names from h2.
val df0 = r1.map(_.split(",")).map{case Array(a,b,c,d,e,f,g,h,i,j,k,l) => 
(a,b.toInt,c,d,e,f.toInt,g,h,i,j.toInt,k.toInt,l.toInt)}.toDF(h2:_*)
df0.take(1)

In [ ]:
val f2 = "rev-devices.csv"
val file2 = sc.textFile(local2(f2).toString())
U.printClass(file2)

In [ ]:
// But error results here if file does not exist
// Or returns empty array if it is empty
val lens = file.map(s => s.length)
file.take(5)
lens.take(5)

In [ ]:
val x0 = file.take(1)

// Some arbitrary file processing - append a number to each line
val pairs = file.map(s => (s, 911))
val counts = pairs.reduceByKey((a, b) => a + b)

In [ ]:
val counts1 = counts.repartition(1)

U.rmdir("counts1")
counts1.saveAsTextFile(local2("counts1").toString())

In [ ]:
val pairs = file.map(x => (x.split(",")(0), x))

val pairs1 = pairs.join(pairs)

In [ ]:
// Make some (K, V) tuples

println(x0(0))

val x1 = x0(0).split(",").toSeq

In [ ]:
val df0 = file.map(_.split(",")).map{case Array(a,b,c,d,e,f,g,h,i,j,k,l) => 
(a,b,c,d,e,f,g,h,i,j,k,l)}.toDF(x1:_*)

In [ ]:
// The x1:_* is to be preferred to this

// val fileToDf = file.map(_.split(",")).map{case Array(a,b,c,d,e,f,g,h,i,j,k,l) => 
// (a,b,c,d,e,f,g,h,i,j,k,l)}.toDF("user_id", "birth_year", "country", "city", "created_date", "user_settings_crypto_unlocked", "plan", "attributes_notifications_marketing_push", "attributes_notifications_marketing_email", "num_contacts", "num_referrals", "num_successful_referrals")

In [ ]:
val df0 = file.map(_.split(",")).map{case Array(a,b,c,d,e,f,g,h,i,j,k,l) => 
(a,b.toInt,c,d,e,f,g,h,i,j,k,l)}.toDF(x1:_*)

In [ ]:
fileToDf.show(3)

In [ ]:
file.map(_.split(",")).take(1)

In [ ]:
val df1 = file.subtract(sc.parallelize(file.take(1)))

In [ ]:
U.printClass(sc)

In [ ]:
df1.take(1)

In [ ]:
def split(f1:String, sep:String)(implicit sc: org.apache.spark.SparkContext) : org.apache.spark.rdd.RDD[String] = {
    val f = sc.textFile(f1)
    return f
}

In [ ]:
split(local2(f1).toString(), ",")(sc)

In [ ]:
U.printClass(sc)

## MLLib

Using LDA.

https://medium.com/analytics-vidhya/distributed-topic-modelling-using-spark-nlp-and-spark-mllib-lda-6db3f06a4da3